In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import xml.etree.ElementTree as ET
from tqdm import tqdm
import matplotlib.pyplot as plt

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Memory management
torch.cuda.empty_cache()

# Configuration
config = {
    'image_size': (256, 256),   # Reduced from 512x512 to save memory
    'batch_size': 4,            # Smaller batch size
    'num_workers': 2,           # Adjust based on your system
    'learning_rate': 3e-4,
    'epochs': 25,
    'max_boxes': 5,             # Reduced from 10 to save memory
    'accumulation_steps': 4     # For gradient accumulation (effectively batch_size * accumulation_steps)
}

# Dataset Paths - Replace with your paths
IMAGE_DIR = "/content/drive/MyDrive/DataSet/IMAGES"
ANNOTATION_DIR = "/content/drive/MyDrive/DataSet/XML"

# Function to parse XML and extract bounding boxes
def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    boxes = []
    for obj in root.findall("object"):
        bbox = obj.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        ymin = int(bbox.find("ymin").text)
        xmax = int(bbox.find("xmax").text)
        ymax = int(bbox.find("ymax").text)

        # Ensure valid bounding boxes
        if xmin >= xmax or ymin >= ymax:
            print(f"Warning: Invalid bounding box in {xml_file}")
            continue

        boxes.append([xmin, ymin, xmax, ymax])

    return boxes

# Efficient Dataset Class
class OCRDataset(Dataset):
    def __init__(self, image_folder, annotation_folder, transform=None, max_boxes=5):
        self.image_folder = image_folder
        self.annotation_folder = annotation_folder
        self.image_files = [f for f in os.listdir(image_folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
        self.transform = transform
        self.max_boxes = max_boxes

        print(f"Found {len(self.image_files)} images")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_name = self.image_files[idx]
        image_path = os.path.join(self.image_folder, image_name)
        xml_path = os.path.join(self.annotation_folder, os.path.splitext(image_name)[0] + ".xml")

        # Load image more efficiently
        image = Image.open(image_path).convert('RGB')

        # Load bounding boxes
        if os.path.exists(xml_path):
            boxes = parse_xml(xml_path)
        else:
            boxes = []

        # Create box tensor with only necessary boxes
        box_tensor = torch.zeros((self.max_boxes, 4))
        num_boxes = min(len(boxes), self.max_boxes)
        if num_boxes > 0:
            box_tensor[:num_boxes] = torch.tensor(boxes[:num_boxes], dtype=torch.float32)

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        return image, box_tensor

# Memory-Efficient CRNN Model for OCR
class CRNN(nn.Module):
    def __init__(self, num_classes=2):
        super(CRNN, self).__init__()

        # Lighter CNN backbone
        self.cnn = nn.Sequential(
            # Layer 1
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Layer 2
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Layer 3
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Layer 4
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        # Calculate output size after CNN
        cnn_output_height = config['image_size'][0] // 16  # After 4 max-pooling layers
        cnn_output_width = config['image_size'][1] // 16

        # RNN layers for sequence modeling
        self.rnn = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * cnn_output_height * cnn_output_width, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU()
        )

        # Classification head
        self.cls_head = nn.Linear(128, num_classes)

        # Bounding box regression head
        self.bbox_head = nn.Linear(128, 4 * config['max_boxes'])  # 4 coordinates per box

    def forward(self, x):
        # Feature extraction through CNN
        features = self.cnn(x)

        # Sequence modeling through RNN
        rnn_out = self.rnn(features)

        # Classification output
        cls_scores = self.cls_head(rnn_out)

        # Bounding box regression output
        bbox_preds = self.bbox_head(rnn_out)
        bbox_preds = bbox_preds.view(-1, config['max_boxes'], 4)

        return cls_scores, bbox_preds

# Define transformations
transform = transforms.Compose([
    transforms.Resize(config['image_size']),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Metrics for evaluation
class OCRMetrics:
    def __init__(self):
        self.reset()

    def reset(self):
        self.total_loss = 0
        self.cls_loss = 0
        self.bbox_loss = 0
        self.samples = 0
        self.iou_scores = []

    def update(self, loss, cls_loss, bbox_loss, pred_boxes, target_boxes, batch_size):
        self.total_loss += loss.item() * batch_size
        self.cls_loss += cls_loss.item() * batch_size
        self.bbox_loss += bbox_loss.item() * batch_size
        self.samples += batch_size

        # Calculate IoU for valid boxes
        for i in range(batch_size):
            for j in range(config['max_boxes']):
                if torch.all(target_boxes[i, j] > 0):  # If target box exists
                    iou = self.calculate_iou(
                        pred_boxes[i, j].detach().cpu(),
                        target_boxes[i, j].detach().cpu()
                    )
                    self.iou_scores.append(iou)

    def calculate_iou(self, box1, box2):
        # Calculate intersection over union
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])

        # Check if there is an intersection
        if x2 < x1 or y2 < y1:
            return 0.0

        intersection = (x2 - x1) * (y2 - y1)

        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])

        union = area1 + area2 - intersection

        return intersection / union if union > 0 else 0.0

    def get_metrics(self):
        avg_loss = self.total_loss / self.samples if self.samples > 0 else 0
        avg_cls_loss = self.cls_loss / self.samples if self.samples > 0 else 0
        avg_bbox_loss = self.bbox_loss / self.samples if self.samples > 0 else 0
        avg_iou = sum(self.iou_scores) / len(self.iou_scores) if self.iou_scores else 0

        return {
            'loss': avg_loss,
            'cls_loss': avg_cls_loss,
            'bbox_loss': avg_bbox_loss,
            'iou': avg_iou
        }

# Function to train the model
def train_model(model, train_loader, criterion_cls, criterion_bbox, optimizer, metrics):
    model.train()
    metrics.reset()

    # Set up progress bar
    progress_bar = tqdm(total=len(train_loader), desc="Training")

    # Initialize gradients
    optimizer.zero_grad()

    for batch_idx, (images, targets) in enumerate(train_loader):
        images = images.to(device)
        targets = targets.to(device)

        # Forward pass
        cls_scores, bbox_preds = model(images)

        # Simple targets: 1 if there's at least one box
        has_text = torch.tensor([
            1 if torch.any(targets[i, :, 0] > 0) else 0
            for i in range(targets.size(0))
        ], dtype=torch.long).to(device)

        # Calculate losses
        cls_loss = criterion_cls(cls_scores, has_text)
        bbox_loss = criterion_bbox(bbox_preds, targets)
        loss = cls_loss + bbox_loss

        # Backward pass with gradient accumulation
        loss = loss / config['accumulation_steps']
        loss.backward()

        # Update weights every accumulation_steps
        if (batch_idx + 1) % config['accumulation_steps'] == 0 or (batch_idx + 1) == len(train_loader):
            optimizer.step()
            optimizer.zero_grad()

        # Update metrics
        metrics.update(loss * config['accumulation_steps'], cls_loss, bbox_loss,
                      bbox_preds, targets, images.size(0))

        # Update progress bar
        progress_bar.update(1)
        if (batch_idx + 1) % 10 == 0:
            metrics_dict = metrics.get_metrics()
            progress_bar.set_postfix({
                'loss': f"{metrics_dict['loss']:.4f}",
                'iou': f"{metrics_dict['iou']:.4f}"
            })

    progress_bar.close()
    return metrics.get_metrics()

# Function to validate the model
def validate_model(model, val_loader, criterion_cls, criterion_bbox, metrics):
    model.eval()
    metrics.reset()

    with torch.no_grad():
        progress_bar = tqdm(total=len(val_loader), desc="Validation")

        for images, targets in val_loader:
            images = images.to(device)
            targets = targets.to(device)

            # Forward pass
            cls_scores, bbox_preds = model(images)

            # Simple targets: 1 if there's at least one box
            has_text = torch.tensor([
                1 if torch.any(targets[i, :, 0] > 0) else 0
                for i in range(targets.size(0))
            ], dtype=torch.long).to(device)

            # Calculate losses
            cls_loss = criterion_cls(cls_scores, has_text)
            bbox_loss = criterion_bbox(bbox_preds, targets)
            loss = cls_loss + bbox_loss

            # Update metrics
            metrics.update(loss, cls_loss, bbox_loss, bbox_preds, targets, images.size(0))

            # Update progress bar
            progress_bar.update(1)

        progress_bar.close()

    return metrics.get_metrics()

# Function to visualize predictions
def visualize_predictions(model, test_loader, num_samples=5):
    model.eval()

    # Get samples
    samples = []
    with torch.no_grad():
        for images, targets in test_loader:
            for i in range(min(images.size(0), num_samples - len(samples))):
                samples.append((images[i], targets[i]))
            if len(samples) >= num_samples:
                break

    # Make predictions and visualize
    fig, axes = plt.subplots(len(samples), 2, figsize=(12, 4 * len(samples)))

    with torch.no_grad():
        for i, (image, target) in enumerate(samples):
            # Original image with target boxes
            img = image.permute(1, 2, 0).numpy()
            img = (img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406]))
            img = np.clip(img, 0, 1)

            axes[i, 0].imshow(img)
            axes[i, 0].set_title("Ground Truth")

            # Draw target boxes
            for j in range(config['max_boxes']):
                if torch.all(target[j] > 0):
                    x1, y1, x2, y2 = target[j].cpu().numpy()
                    # Scale coordinates to image size
                    x1 = x1 * config['image_size'][1] / 1000
                    y1 = y1 * config['image_size'][0] / 1000
                    x2 = x2 * config['image_size'][1] / 1000
                    y2 = y2 * config['image_size'][0] / 1000
                    rect = plt.Rectangle((x1, y1), x2-x1, y2-y1,
                                        fill=False, edgecolor='green', linewidth=2)
                    axes[i, 0].add_patch(rect)

            # Predicted boxes
            image_tensor = image.unsqueeze(0).to(device)
            cls_scores, bbox_preds = model(image_tensor)

            axes[i, 1].imshow(img)
            axes[i, 1].set_title("Predictions")

            # Draw predicted boxes
            for j in range(config['max_boxes']):
                box = bbox_preds[0, j].cpu().numpy()
                x1, y1, x2, y2 = box

                # Scale coordinates to image size
                x1 = x1 * config['image_size'][1] / 1000
                y1 = y1 * config['image_size'][0] / 1000
                x2 = x2 * config['image_size'][1] / 1000
                y2 = y2 * config['image_size'][0] / 1000

                if x1 < x2 and y1 < y2:  # Ensure valid box
                    rect = plt.Rectangle((x1, y1), x2-x1, y2-y1,
                                        fill=False, edgecolor='red', linewidth=2)
                    axes[i, 1].add_patch(rect)

    plt.tight_layout()
    plt.savefig('ocr_predictions.png')
    plt.close()
    return 'ocr_predictions.png'

# Main training loop
def main():
    print("Starting OCR model training...")

    # Create datasets
    dataset = OCRDataset(IMAGE_DIR, ANNOTATION_DIR, transform=transform, max_boxes=config['max_boxes'])

    # Split dataset
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=config['batch_size'],
        shuffle=True,
        num_workers=config['num_workers'],
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=config['batch_size'],
        shuffle=False,
        num_workers=config['num_workers'],
        pin_memory=True
    )

    print(f"Created data loaders. Train: {len(train_loader)} batches, Val: {len(val_loader)} batches")

    # Initialize model
    model = CRNN(num_classes=2).to(device)

    # Print model summary
    print(model)

    # Memory-efficient optimizer and scheduler
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5)

    # Loss functions
    criterion_cls = nn.CrossEntropyLoss()
    criterion_bbox = nn.SmoothL1Loss()

    # Training metrics
    train_metrics = OCRMetrics()
    val_metrics = OCRMetrics()

    # Training history
    history = {
        'train_loss': [],
        'train_iou': [],
        'val_loss': [],
        'val_iou': []
    }

    # Training loop
    for epoch in range(config['epochs']):
        print(f"\nEpoch {epoch+1}/{config['epochs']}")

        # Train
        train_metrics_dict = train_model(
            model, train_loader, criterion_cls, criterion_bbox, optimizer, train_metrics
        )

        # Validate
        val_metrics_dict = validate_model(
            model, val_loader, criterion_cls, criterion_bbox, val_metrics
        )

        # Update learning rate
        scheduler.step(val_metrics_dict['loss'])

        # Save metrics
        history['train_loss'].append(train_metrics_dict['loss'])
        history['train_iou'].append(train_metrics_dict['iou'])
        history['val_loss'].append(val_metrics_dict['loss'])
        history['val_iou'].append(val_metrics_dict['iou'])

        # Print metrics
        print(f"Train Loss: {train_metrics_dict['loss']:.4f}, Train IoU: {train_metrics_dict['iou']:.4f}")
        print(f"Val Loss: {val_metrics_dict['loss']:.4f}, Val IoU: {val_metrics_dict['iou']:.4f}")

        # Save model checkpoint
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_metrics_dict['loss'],
            'val_loss': val_metrics_dict['loss'],
        }, f'ocr_model_epoch_{epoch+1}.pth')

    # Plot training history
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history['train_iou'], label='Train IoU')
    plt.plot(history['val_iou'], label='Val IoU')
    plt.title('IoU Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('IoU')
    plt.legend()

    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

    # Visualize predictions
    visualize_predictions(model, val_loader)

    print("Training completed!")

    # Save final model
    torch.save(model.state_dict(), 'ocr_model_final.pth')

    return model

# For working with PDF files
def extract_images_from_pdf(pdf_path, output_dir):
    """Extract images from PDF file for processing"""
    from pdf2image import convert_from_path
    import os

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Convert PDF to images
    pages = convert_from_pdf(pdf_path, dpi=300)

    # Save images
    image_paths = []
    for i, page in enumerate(pages):
        output_path = os.path.join(output_dir, f'page_{i+1}.jpg')
        page.save(output_path, 'JPEG')
        image_paths.append(output_path)

    return image_paths

# Function for text extraction from images
def extract_text_from_images(model, image_paths):
    """Extract text from images using trained model"""
    model.eval()

    results = []
    for image_path in tqdm(image_paths, desc="Extracting text"):
        # Load image
        image = Image.open(image_path).convert('RGB')
        transform_image = transform(image).unsqueeze(0).to(device)

        # Get predictions
        with torch.no_grad():
            cls_scores, bbox_preds = model(transform_image)

        # Get text regions
        text_regions = []
        for i in range(config['max_boxes']):
            box = bbox_preds[0, i].cpu().numpy()
            x1, y1, x2, y2 = box

            # Scale coordinates to original image size
            img_width, img_height = image.size
            x1 = int(x1 * img_width / 1000)
            y1 = int(y1 * img_height / 1000)
            x2 = int(x2 * img_width / 1000)
            y2 = int(y2 * img_height / 1000)

            # Ensure valid coordinates
            if x1 < x2 and y1 < y2 and x1 >= 0 and y1 >= 0 and x2 <= img_width and y2 <= img_height:
                text_regions.append((x1, y1, x2, y2))

        # Add to results
        results.append({
            'image_path': image_path,
            'text_regions': text_regions
        })

    return results

# Function to perform OCR on detected text regions
def perform_ocr_on_regions(results, output_file):
    """Perform OCR on detected text regions using Tesseract"""
    import pytesseract
    from PIL import Image

    with open(output_file, 'w', encoding='utf-8') as f:
        for result in tqdm(results, desc="Performing OCR"):
            image_path = result['image_path']
            text_regions = result['text_regions']

            # Open image
            image = Image.open(image_path).convert('RGB')

            # Extract text from each region
            page_text = []
            for region in text_regions:
                x1, y1, x2, y2 = region
                region_image = image.crop((x1, y1, x2, y2))

                # Perform OCR
                text = pytesseract.image_to_string(region_image, lang='eng')
                page_text.append(text.strip())

            # Write to file
            f.write(f"=== {os.path.basename(image_path)} ===\n\n")
            f.write('\n\n'.join(page_text))
            f.write('\n\n')

    return output_file

# Complete OCR pipeline
def ocr_pipeline(pdf_path, output_dir):
    """Complete OCR pipeline from PDF to text"""
    # Extract images from PDF
    image_paths = extract_images_from_pdf(pdf_path, output_dir)

    # Load trained model
    model = CRNN(num_classes=2).to(device)
    model.load_state_dict(torch.load('ocr_model_final.pth'))

    # Extract text regions
    results = extract_text_from_images(model, image_paths)

    # Perform OCR on regions
    output_file = os.path.join(output_dir, 'ocr_output.txt')
    perform_ocr_on_regions(results, output_file)

    return output_file

# Evaluation metrics specific to OCR tasks
def calculate_ocr_metrics(ground_truth_file, predicted_file):
    """Calculate OCR-specific metrics"""
    import difflib

    # Read files
    with open(ground_truth_file, 'r', encoding='utf-8') as f:
        ground_truth = f.read()

    with open(predicted_file, 'r', encoding='utf-8') as f:
        predicted = f.read()

    # Normalize text
    ground_truth = ' '.join(ground_truth.split())
    predicted = ' '.join(predicted.split())

    # Calculate character error rate (CER)
    total_chars = len(ground_truth)
    diff = difflib.SequenceMatcher(None, ground_truth, predicted)
    matches = sum(end - start for i, j, start, end in diff.get_matching_blocks())
    cer = 1 - (matches / total_chars)

    # Calculate word error rate (WER)
    ground_truth_words = ground_truth.split()
    predicted_words = predicted.split()

    diff = difflib.SequenceMatcher(None, ground_truth_words, predicted_words)
    matches = sum(end - start for i, j, start, end in diff.get_matching_blocks())
    wer = 1 - (matches / len(ground_truth_words))

    return {
        'CER': cer,
        'WER': wer,
        'Accuracy': 1 - cer
    }

if __name__ == "__main__":
    # Train model
    model = main()

Using device: cuda
Starting OCR model training...
Found 430 images
Created data loaders. Train: 86 batches, Val: 22 batches
CRNN(
  (cnn): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (rnn): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=65


Training: 100%|██████████| 86/86 [01:53<00:00,  1.32s/it, loss=999.5243, iou=0.0023]

Validation: 100%|██████████| 22/22 [00:31<00:00,  1.45s/it]


Train Loss: 971.7064, Train IoU: 0.0047
Val Loss: 493.2337, Val IoU: 0.0766

Epoch 2/25



Training: 100%|██████████| 86/86 [00:26<00:00,  3.25it/s, loss=339.6455, iou=0.0331]

Validation: 100%|██████████| 22/22 [00:05<00:00,  3.81it/s]


Train Loss: 330.4337, Train IoU: 0.0364
Val Loss: 202.7890, Val IoU: 0.0585

Epoch 3/25


...

Train Loss: 108.1704, Train IoU: 0.1578
Val Loss: 87.6627, Val IoU: 0.2058

Epoch 24/25



Training: 100%|██████████| 86/86 [00:26<00:00,  3.25it/s, loss=106.2946, iou=0.1656]

Validation: 100%|██████████| 22/22 [00:05<00:00,  3.79it/s]


Train Loss: 106.6053, Train IoU: 0.1645
Val Loss: 75.2697, Val IoU: 0.2987

Epoch 25/25



Training: 100%|██████████| 86/86 [00:26<00:00,  3.19it/s, loss=107.0994, iou=0.1679]

Validation: 100%|██████████| 22/22 [00:07<00:00,  3.00it/s]


Train Loss: 106.2126, Train IoU: 0.1726
Val Loss: 62.4008, Val IoU: 0.4123
Training completed!
